# Problems with this notebook
1. It does nothing intelligent with the days but simply whacks them all into one model;
2. It uses atypical seasonal data (Christmas/New Year);
3. It puts the same result for every predicted day in the output (just puts in a general page prediction);
4. Please add more...

# Next steps
My intuition is that we need to know what pages in the key we're dealing with and what dates. I recommend splitting (as I have at the moment) out the first column into two new columns giving us that information.

Then, I think we put non-date-specific page predictions into each row and use them to make a new set of preditions based on them *and* the date to give us a final Visits column.

# Current score
86.1 already a big improvement on previous, but hasn't changed my place as a load of new people have jumped in with slightly lower scores.

### Tips
This can be uploaded to Kaggle (using the up cloud button) and will work there once you add this competition's dataset, as the data is also stored in ../input/. You can download it to work on your own machine tinker with output etc. and, of course, to commit.

In [1]:
# Handy shizzle
import pandas as pd
from sklearn.linear_model import LinearRegression

## With the whole thing?!

In [2]:
# Get the data and tidy it up
df = pd.read_csv('../input/train_1.csv')
df = df.drop(df.columns[1:-9], axis=1).fillna(0)

In [3]:
# Add spider variable
df['spider'] = 0
df.ix[df.Page.str.contains('spider'), 'spider'] = 1
# Add desktop variable
df['desktop'] = 0
df.ix[df.Page.str.contains('desktop'), 'desktop'] = 1
df.head()

In [26]:
df.ix[df.Page.str.contains('龙生九子_zh.wikipedia.org_mobile-web_all-agents')]

In [4]:
# Have a punt!
dfLR = LinearRegression()
x = df.drop(['Page', '2016-12-31'], axis=1)
y = df['2016-12-31']

In [5]:
dfLR.fit(x, y)

In [6]:
dfLR.score(x, y)

In [7]:
results = dfLR.predict(x)
results[0:10]

In [8]:
results = [max(0, num) for num in results]
results[0:10]

# Try a submission!

In [9]:
import pandas as pd
ss_df = pd.read_csv('../input/key_1.csv')
ss_df.head()

## Transferring data into correct rows in key

In [10]:
# The plan is to cut dates out of names
# and copy results by name from df into here
# First add results to df.
pages_with_results = pd.concat([df, pd.DataFrame(results, columns=['Visits'])], axis=1)
pages_with_results.head()

In [11]:
# Get the page names
ss_df['Page'] = [name.split('_2017')[0] for name in ss_df['Page'].values]

In [12]:

ss_df['Visits'] = 0
ss_df

In [13]:
# makes it easier to map visits from their page
ss_df['Visits'] = ss_df['Page']
ss_df.head()

In [14]:
#ss_df['Visits'] = [get_results_for_page(page) for page in ss_df['Page'].values]
ss_df

In [15]:
# df of results indexed by page--makes mapping quicker
page_indexed_visits = pages_with_results.set_index(df['Page'])
page_indexed_visits.ix['!vote_en.wikipedia.org_all-access_all-agents', 'Visits']

In [16]:
# stops fatal error on missing data
def get_page_visits (page):
    try:
        return page_indexed_visits.ix[page, 'Visits']
    except KeyError:
        return 0
    
get_page_visits('龙生九子_zh.wikipedia.org_mobile-web_all-agents')

In [17]:
ss_df['Visits'] = ss_df.Visits.map(get_page_visits)

In [18]:
ss_df.tail()

## Export!

In [21]:
ss_df.drop(['Page'], axis=1).to_csv('PWM_ss.csv', index=False, header=True)

In [23]:
test = pd.read_csv('PWM_ss.csv')

In [24]:
test.head()

In [25]:
test.tail()